## MNIST机器学习入门

tensorfly.cn 的链接是无法打开，[官方源码github](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/mnist)

In [2]:
### inputdata
import gzip
import os
import tempfile

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [3]:
## 手动下载之后不要解压！！！否则读取失败
mnist = read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print(mnist.train.images.shape, mnist.train.labels.shape)

(55000, 784) (55000, 10)


训练集55000张图片，每张图片28x28 = 784像素。使用one-hot 把lable转换成长度为10的向量，所有lable集是55000x10的矩阵

### Softmax回归

每一张图片都表示一个数字，从0到9。我们希望得到给定图片代表每个数字的概率。比如说，我们的模型可能推测一张包含9的图片代表数字9的概率是80%但是判断它是8的概率是5%（因为8和9都有上半部分的小圆），然后给予它代表其他数字的概率更小的值。

使用x表示图片像素特征矩阵，b表示偏移常数矩阵，W表示像素特征权重矩阵，y表示label矩阵
xW+b=y

In [105]:
###x不是一个特定的值，而是一个占位符placeholder
### 我们希望能够输入任意数量的MNIST图像，每一张图展平成784维的向量。
### 这个张量的形状是[None，784 ]。（这里的None表示此张量的第一个维度可以是任何长度的。）
x = tf.placeholder("float", [None, 784])

In [6]:
##我们赋予tf.Variable不同的初值来创建不同的Variable：
##我们都用全为零的张量来初始化W和b。因为我们要学习W和b的值，它们的初值可以随意设置。
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [7]:
## y表示预测值
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [8]:
## 表示实际值
y_ = tf.placeholder("float", [None,10])

In [9]:
## 交叉熵 用于评估预测值和实际值的差
cross_entropy = -tf.reduce_sum(y_*tf.log(y))


In [10]:
## 使用梯度下降法最小化交叉熵,学习率不能设置太大，否则会无法收敛
train_step = tf.train.GradientDescentOptimizer(0.002).minimize(cross_entropy)

In [11]:
init = tf.global_variables_initializer()

In [12]:
sess = tf.Session()
sess.run(init)

In [13]:
## 循环1000次，每次使用100个训练数据
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(300)
    s = sess.run([train_step,cross_entropy], feed_dict={x: batch_xs, y_: batch_ys})

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [15]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [16]:
print(sess.run(accuracy, feed_dict={x: mnist.train.images, y_: mnist.train.labels}))

0.9199091


In [17]:
correct_prediction 

<tf.Tensor 'Equal:0' shape=(?,) dtype=bool>

### 性能提升

我们需要创建大量的权重和偏置项。这个模型中的权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度。

由于我们使用的是ReLU神经元，因此比较好的做法是用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为0的问题（dead neurons）

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### 卷积和池化

#### 卷积核计算输入输出

[参考文章](https://blog.csdn.net/qq_17272679/article/details/79591540)

x 的宽度x_width,高度 x_height, 卷积核宽度W_width,W_height

padding = VALID , 直接不考虑边界问题。输出图片宽度计算: (x_width-W_width)/strides+1

padding = SAVE 考虑边界填充。

填充数目paddingnum,strides>1，填充数目为W_width-x_width%strides 

如果strides=1 填充数 x_width-1。这种情况下输出大小同输入大小

In [94]:
## 卷积计算
## 步长1，1，stride首末两个固定1，中间表示每个维度步长，padding='SAVE'考虑边界，不足用0填充。'VALID' 不考虑
def conv2d(x, W):
    ## 输入输出维度计算
    ## x 的宽度x_width,高度 x_height, 卷积核宽度W_width,W_height
    ## padding = VALID , 直接不考虑边界问题。输出图片宽度计算: (x_width-W_width)/strides+1
    ## padding = SAVE 考虑边界填充。
    ## 填充数目paddingnum,strides>1，填充数目为W_width-x_width%strides 
    ## 如果strides=1 填充数 x_width-1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

## 池化，池化层夹在连续的卷积层中间， 用于压缩数据和参数的量，减小过拟合。
##池化层用的方法有Max pooling 和 average pooling，而实际用的较多的是Max pooling。
## 对于每个2*2的窗口选出最大的数作为输出矩阵的相应元素的值，比如输入矩阵第一个2*2窗口中最大的数是6，那么输出矩阵的第一个元素就是6，如此类推。
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [106]:
## 计算第一层卷积
## 卷积在每个5x5的patch中算出32个特征(32个卷积核计算)。卷积的权重张量形状是[5, 5, 1, 32]，（32是自定义的）
## 前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目。 而对于每一个输出通道都有一个对应的偏置量。
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#

In [107]:
x_image = tf.reshape(x, [-1,28,28,1])

In [108]:
## 使用发射函数获得第一层卷积层输出

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [109]:
## 对卷积层输出进行池化
h_pool1 = max_pool_2x2(h_conv1)

In [110]:
## 计算第二层卷积（64卷积核个数自定义）
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

两层卷积池化之后，图像大小转换成7*7 第二层卷积输出64个特征

全连接层

In [111]:


W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [112]:
## dropout 减少过拟合
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [113]:
## 输出层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [116]:
with tf.Session() as sess:
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
      batch = mnist.train.next_batch(50)
      if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.04
step 100, training accuracy 0.82
step 200, training accuracy 0.92
step 300, training accuracy 0.88
step 400, training accuracy 0.9
step 500, training accuracy 0.98
step 600, training accuracy 0.88
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.96
step 1100, training accuracy 0.96
step 1200, training accuracy 0.98
step 1300, training accuracy 0.9
step 1400, training accuracy 0.88
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.94
test accuracy 0.9756
